*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
#get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
df_train = pd.read_table('train-data.tsv', header=None)
df_test = pd.read_table('valid-data.tsv', header=None)

In [ ]:
ham_msg = df_train[df_train[0] =='ham']
spam_msg = df_train[df_train[0]=='spam']

In [ ]:
ham_msg_df = ham_msg.sample(n = len(spam_msg), random_state = 44)
spam_msg_df = spam_msg
print(ham_msg_df.shape, spam_msg_df.shape)

In [ ]:
#here balanced data, equal amount of ham and spam
train=pd.concat([ham_msg_df,spam_msg_df],axis=0)

In [ ]:
'''
df=pd.concat([df_train,df_test],axis=0)
'''

In [ ]:
'''
import string

def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

df['2']=df[1].apply(lambda x : remove_punct(x))
'''

In [ ]:
'''
### if we use remove_punct function we need to change 1 to 2, in here. 
df["3"] = df[1].apply(lambda x: x.split())
'''

In [ ]:
'''
df["4"] = df["3"].apply(lambda x: " ".join(x))
'''

In [ ]:
'''
df_train=df[:df_train.shape[0]]
df_test=df[df_train.shape[0]:]
'''

In [ ]:
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(train[1],train[0], random_state = 1)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf_word_vectorizer = TfidfVectorizer()
tf_idf_word_vectorizer.fit_transform(train_x)

In [ ]:
train_x_tf_idf_word = tf_idf_word_vectorizer.transform(train_x)
val_x_tf_idf_word = tf_idf_word_vectorizer.transform(val_x)

In [ ]:
val_x_tf_idf_word.shape

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import model_selection
nb = MultinomialNB()
nb_model = nb.fit(train_x_tf_idf_word,train_y)
accuracy = model_selection.cross_val_score(nb_model, 
                                           val_x_tf_idf_word, 
                                           val_y, 
                                           cv = 10).mean()

print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)

In [ ]:
 test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

In [ ]:
test_messages = pd.DataFrame(test_messages)

In [ ]:
test_messages_tf_idf_word = tf_idf_word_vectorizer.transform(test_messages[0])

In [ ]:
nb_model.predict(test_messages_tf_idf_word)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  from sklearn.naive_bayes import MultinomialNB
  from sklearn import model_selection
  nb = MultinomialNB()
  nb_model = nb.fit(train_x_tf_idf_word,train_y)
  accuracy = model_selection.cross_val_score(nb_model, 
                                           val_x_tf_idf_word, 
                                           val_y, 
                                           cv = 10).mean()

  #print("Word-Level TF-IDF Doğruluk Oranı:", accuracy)

  test_messages = pred_text
  test_messages = pd.DataFrame([pred_text])
  test_messages_tf_idf_word = tf_idf_word_vectorizer.transform(test_messages[0])
  
 
  
  prediction =[]
  #prediction.append([nb_model.predict(test_messages_tf_idf_word)])
  

  prediction.append(["heyoo",nb_model.predict(test_messages_tf_idf_word)[0]])
  #prediction = nb_model.predict(test_messages_tf_idf_word)
  #prediction[1] = nb_model.predict(test_messages_tf_idf_word)

  return (prediction[0])


hey = "how are you doing today?"
#pred_text = "how are you doing today?"

deneme = predict_message(test_messages)
print(deneme)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print("prediction verisi: ", prediction)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
